In [107]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import count, explode,col, when, lit, concat_ws, udf, from_json, struct, to_json,window,sum as agg_sum ,count , aggregate,to_date ,dayofweek,hour , minute ,concat
import pandas as pd
from pyspark.sql.pandas.functions import pandas_udf
from pyspark.sql.functions import collect_list, col, when, lit, concat_ws, udf, from_json, struct, to_json,window,sum as agg_sum ,count , aggregate
from pyspark.sql.pandas.functions import pandas_udf, PandasUDFType
from abc import ABC, abstractmethod, abstractclassmethod
from pyspark.sql import DataFrame, SparkSession
from pyspark.sql.functions import col, when, lit, concat_ws, udf, from_json, struct, to_json, window, collect_list ,avg
from pyspark.sql.types import StructType, StructField, StringType, DateType, FloatType
from pipeline.points import TEHRAN_POINTS ,Point

from pyspark.sql.types import StructType, StructField, StringType, DateType, FloatType ,ArrayType ,MapType ,IntegerType ,Union
spark = SparkSession.builder.getOrCreate()


In [102]:
df = spark.read.csv("./storage/sample_data2.txt",header=True)
df.show()

+--------+---------+---------+--------------------+
|driverId|      lat|      lng|                time|
+--------+---------+---------+--------------------+
|     180|35.716124|51.439540|2020-10-20 12:54:...|
|     154|35.674238|51.324752|2020-10-20 12:54:...|
|     118|35.660226|51.317367|2020-10-20 12:54:...|
|     194|35.725052|51.463792|2020-10-20 12:54:...|
|     132|35.758336|51.448136|2020-10-20 12:54:...|
|     106|35.746166|51.340506|2020-10-20 12:54:...|
|     113|35.721141|51.390772|2020-10-20 12:54:...|
|     172|35.671891|51.458438|2020-10-20 12:54:...|
|     172|35.728834|51.375421|2020-10-20 12:54:...|
|     113|35.721141|51.390772|2020-10-20 12:54:...|
|     173|35.671891|51.458438|2020-10-20 12:54:...|
|     173|35.728834|51.375421|2020-10-20 12:54:...|
|     173|35.728834|51.375421|2020-10-20 12:54:...|
+--------+---------+---------+--------------------+



In [103]:
# SELECT DATEPART(WEEKDAY, GETDATE())
df =df.withColumn("dayOfWeek_Hour",concat(dayofweek(col("time")),lit("_") ,hour(col("time"))))
df.show()

+--------+---------+---------+--------------------+--------------+
|driverId|      lat|      lng|                time|dayOfWeek_Hour|
+--------+---------+---------+--------------------+--------------+
|     180|35.716124|51.439540|2020-10-20 12:54:...|          3_12|
|     154|35.674238|51.324752|2020-10-20 12:54:...|          3_12|
|     118|35.660226|51.317367|2020-10-20 12:54:...|          3_12|
|     194|35.725052|51.463792|2020-10-20 12:54:...|          3_12|
|     132|35.758336|51.448136|2020-10-20 12:54:...|          3_12|
|     106|35.746166|51.340506|2020-10-20 12:54:...|          3_12|
|     113|35.721141|51.390772|2020-10-20 12:54:...|          3_12|
|     172|35.671891|51.458438|2020-10-20 12:54:...|          3_12|
|     172|35.728834|51.375421|2020-10-20 12:54:...|          3_12|
|     113|35.721141|51.390772|2020-10-20 12:54:...|          3_12|
|     173|35.671891|51.458438|2020-10-20 12:54:...|          3_12|
|     173|35.728834|51.375421|2020-10-20 12:54:...|          3

In [106]:



@pandas_udf(StringType(), None)
def assign_closet_point(lat_series: pd.Series, long_series: pd.Series) -> pd.Series:
        df = pd.DataFrame({"lat": lat_series, "long": long_series})
        print(lat_series)
        df["assigned_point"] = df.apply(lambda item: min(TEHRAN_POINTS, key=lambda p: Point.distance(p, Point(item["lat"], item["long"], ''))).label, axis=1)
        return df["assigned_point"]

    
@udf(returnType=ArrayType(
        StructType([
            StructField("p1", StringType()),
            StructField("p2", StringType()),
            StructField("edge", IntegerType()),
        ])))
def generate_graph_edges(list_of_point_labels: list):
        weights = {label: list_of_point_labels.count(label) for label in set(list_of_point_labels)}
        result = []
        last_point = list_of_point_labels[0]

        for p in list_of_point_labels[1:]:
            if p != last_point:
                result.append({"p1": last_point, "p2": p, "edge": weights[last_point]})
                last_point = p

        return result
    
df = df.withColumn("assigned_point", assign_closet_point(col("lat"), col('lng')))
x= df.orderBy("time").groupBy("driverId","dayOfWeek_Hour").agg(collect_list(col("assigned_point")).alias("assigned_point"))
x = x.withColumn("Y" ,generate_graph_edges(col("assigned_point")))
x= x.withColumn("u", explode(col("Y"))).select("*" ,"u.*" )
x.show()







0     35.716124
1     35.674238
2     35.660226
3     35.725052
4     35.758336
5     35.746166
6     35.721141
7     35.671891
8     35.728834
9     35.721141
10    35.671891
11    35.728834
12    35.728834
Name: _0, dtype: object
0     35.716124                                                                 
1     35.674238
2     35.660226
3     35.725052
4     35.758336
5     35.746166
6     35.721141
7     35.671891
8     35.728834
9     35.721141
10    35.671891
11    35.728834
12    35.728834
Name: _0, dtype: object


+--------+--------------+--------------------+--------------------+--------------------+--------+----------+----+
|driverId|dayOfWeek_Hour|      assigned_point|                   Y|                   u|      p1|        p2|edge|
+--------+--------------+--------------------+--------------------+--------------------+--------+----------+----+
|     172|          3_12|[POINT_90, POINT_...|[{POINT_90, POINT...|{POINT_90, POINT_...|POINT_90|POINT_5954|   1|
|     173|          3_12|[POINT_90, POINT_...|[{POINT_90, POINT...|{POINT_90, POINT_...|POINT_90|POINT_5954|   1|
+--------+--------------+--------------------+--------------------+--------------------+--------+----------+----+



In [109]:
final_df = x.groupBy("dayOfWeek_Hour" ,"p1","p2").agg(avg(col("edge").alias("edge_on_avg"))).show()

0     35.716124
1     35.674238
2     35.660226
3     35.725052
4     35.758336
5     35.746166
6     35.721141
7     35.671891
8     35.728834
9     35.721141
10    35.671891
11    35.728834
12    35.728834
Name: _0, dtype: object
0     35.716124                                                                 
1     35.674238
2     35.660226
3     35.725052
4     35.758336
5     35.746166
6     35.721141
7     35.671891
8     35.728834
9     35.721141
10    35.671891
11    35.728834
12    35.728834
Name: _0, dtype: object


+--------------+--------+----------+------------------------+
|dayOfWeek_Hour|      p1|        p2|avg(edge AS edge_on_avg)|
+--------------+--------+----------+------------------------+
|          3_12|POINT_90|POINT_5954|                     1.0|
+--------------+--------+----------+------------------------+



2